###### fake agent擋阻擋用api

In [1]:
import sys
sys.path.append("/Users/Apple/pyds/lib/python3.8/site-packages")
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import xlrd

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

#右鍵操作
from selenium.webdriver import ActionChains#模擬鼠標操作
import pyautogui#鍵盤、鼠標操作

In [176]:
df1=pd.read_csv("新北市醫院.csv",index_col=False)

In [ ]:
df1

### 直轄市才可用

In [178]:
city_county1=df1["地址"].str.split("區",expand=True)[0].str.split("市",expand=True)
new1 = df1["地址"].str.split("區",expand=True)[1]#Series

In [180]:
#市/區的處理
city1=[]
country1=[]
for  i , z in zip(city_county1[0],city_county1[1]):
    x=i+"市"
    y=z+"區"
    city1.append(x)
    country1.append(y)
all1 = np.transpose(df1["地址"].values).tolist() 

In [181]:
#以切
stree1=[]
number1=[]
error1=[]#重新調整迴圈
for  i in range(len(new1)) :
    if "里" in new1[i]:
        stree_number=new1[i].split("里",expand=True)[1]
        if "路" and "段" in stree_number:
            x=stree_number.split("段")
            ystree=x[0]+"段"
            znumber=x[1].split("號")[0]
        
        
        elif "路" in stree_number:
            x=stree_number.split("路")
            ystree=x[0]+"路"
            znumber=x[1].split("號")[0]
        
        elif "段" in stree_number:
            x=stree_number.split("段")
            ystree=x[0]+"段"
            znumber=x[1].split("號")[0]
        
        elif "街" in stree_number:
            x=stree_number.split("街")
            ystree=x[0]+"街"
            znumber=x[1].split("號")[0]
        else:
            k1=stree_number

    else:
        if "路" and "段" in new1[i]:
            x=new1[i].split("段")
            ystree=x[0]+"段"
            znumber=x[1].split("號")[0]
        elif "路" in new1[i]:
            x=new1[i].split("路")
            ystree=x[0]+"路"
            znumber=x[1].split("號")[0]
        
        elif "段" in new1[i]:
            x=new1[i].split("段")
            ystree=x[0]+"段"
            znumber=x[1].split("號")[0]
        
        elif "街" in new1[i]:
            x=new1[i].split("街")
            ystree=x[0]+"街"
            znumber=x[1].split("號")[0]
        elif "腳" in new1[i]:
            f1=new1[i].split("腳")
            ystree=f1[0]+"腳"
            znumber=f1[1].split("號")[0]
        elif "山" in new1[i]:
            f1=new1[i].split("山")
            ystree=f1[0]+"山"
            znumber=f1[1].split("號")[0]
        else:
            e1=new1[i].split("號")[0]
            print(e1)
               
    stree1.append(ystree)
    number1.append(znumber)
    error1.append(e1)
            
    
    

In [182]:
df1.columns

Index(['醫院', '縣市別', '鄉鎮市區', '地址'], dtype='object')

In [183]:
df2 = pd.DataFrame({"地址":all1,"縣市別":city1,"鄉鎮市區":country1,"道路":stree1,"號":number1})

In [184]:
df2.head()

,地址,縣市別,鄉鎮市區,道路,號
0,臺北市中山區林森北路530號,臺北市,中山區,林森北路,530
1,臺北市中山區中山北路二段92號,臺北市,中山區,中山北路二段,92
2,臺北市萬華區廣州街200號,臺北市,萬華區,廣州街,200
3,臺北市萬華區康定路37號,臺北市,萬華區,康定路,37
4,臺北市萬華區西園路二段270號,臺北市,萬華區,西園路二段,270


In [185]:
lat1=[]#緯度25
log1=[]#經度121

## 進入網頁

In [243]:
options = webdriver.ChromeOptions()
browser =  webdriver.Chrome("/Users/Apple/pyds/chromedriver",options=options)
wait = ui.WebDriverWait(browser,10)#10秒内每隔500毫秒扫描1次页面变化

In [244]:
browser.get("https://easymap.land.moi.gov.tw/R02/Index")

In [245]:
noshowagain=browser.find_element_by_css_selector("#notShowSysinfo_id")
webdriver.ActionChains(browser).move_to_element(noshowagain).click(noshowagain).perform()#不在顯示
browser.find_element_by_xpath("/html/body/div[1]/div/div[1]/div/ul/li[2]/img").click()
time.sleep(2)
Select(browser.find_element_by_id("doorPlateTypeId")).select_by_value("B")#value=B 戶政門牌
time.sleep(1)
Select(browser.find_element_by_id("select_city_id1")).select_by_visible_text(city1[0])#縣市
time.sleep(1)

In [251]:
way1=[]
for i ,j ,k in zip(country1[24:],stree1[24:],number1[24:]):
    #鄉鎮市區
    Select(browser.find_element_by_id("select_town_id1")).select_by_visible_text(i)
    time.sleep(3)
    #路名
    wait.until(lambda browser: browser.find_element_by_id("select_road_id"))
    Select(browser.find_element_by_id("select_road_id")).select_by_visible_text(j)
    time.sleep(3)
    #輸入縣市
    code=browser.find_element_by_id("doorNoId")
    code.clear()
    code.send_keys(k)
    time.sleep(1)
    #點擊搜尋
    wait.until(lambda browser: browser.find_element_by_css_selector("#door_botton"))
    element1=browser.find_element_by_css_selector("#door_botton")#解決無法點擊
    webdriver.ActionChains(browser).move_to_element(element1).click(element1).perform()#解決無法點擊(元素截獲異常，該元素不支援點擊)
    time.sleep(13)
    #右鍵點擊圖標，第二個是座標
    wait.until(lambda browser: browser.find_element_by_css_selector("#OpenLayers\.Layer\.Vector_28_vroot"))
    logo = browser.find_element_by_css_selector("#OpenLayers\.Layer\.Vector_28_vroot")  
    rightClick = ActionChains(browser)  # 實例化ActionChains類
    rightClick.context_click(logo).perform()  
    time.sleep(1)
    pyautogui.typewrite(['down'])  # 選中右鍵菜單中第1個選項
    pyautogui.typewrite(['return']) 
    time.sleep(2)
    y=browser.find_element_by_css_selector("#coordDisplayLonLat").text
    print(y)
    #放入座標
    way1.append(y)
    #點除浮動頁面
    enter1 = browser.find_element_by_css_selector("#btnLeftClickCoordDisplayId")
    browser.execute_script("arguments[0].click();",enter1)#解決javascriptExecption
    time.sleep(10)
    
    


25.04145,121.51720
25.01652,121.52990
25.03561,121.50672
25.04439,121.51877
25.04080,121.51857
25.05030,121.53395
25.05891,121.52257
25.01452,121.54393

25.04314,121.55687
25.03852,121.54704
25.03697,121.55371
25.03658,121.54972
25.03757,121.54531
25.05072,121.50890
25.10522,121.53158
25.09627,121.52030
25.03689,121.49801


In [271]:
#把座標拆分成經緯度
for i in  getway:
    x=i.split(",")
    lat1.append(x[0])
    log1.append(x[1])

In [276]:
df1["緯度"]=lat1
df1["經度"]=log1

In [278]:
df1.to_csv("臺北市醫院經緯度版.csv",index=False)